# Описание исследования

Проводим работу с табличными данными, в которых представлена информация о стартапах, которые функционировали в период с 1980 по 2018 годы. Нужно предсказать, какие из них закроются, а какие нет. Соревнование проводится на популярной платформе Kaggle, что позволит не только применять на практике свои знания в области анализа данных и машинного обучения, но и освоить работу с этой платформой. 

# Цель исследования

- Разработать модель машинного обучения для предсказания продолжения деятельности стартапа. 
- Провести полноценный разведочный анализ и сформировать рекомендации будущим создателям стартапов (какие факторы влияют на успешность стартапа).

# Этапы исследования

- загрузка и ознакомление с данными,
- предварительная обработка,
- полноценный разведочный анализ,
- разработка новых синтетических признаков,
- проверка на мультиколлинеарность,
- отбор финального набора обучающих признаков,
- выбор и обучение моделей,
- итоговая оценка качества предсказания лучшей модели,
- анализ важности ее признаков,
- подготовка отчета по исследованию.

# Описание данных

Датасет состоит из двух файлов: 
- тренировочный набор (около 53к записей) и тестовый набор (около 13к записей). 
- Тренировочный набор содержит целевой признак status, указывающий на то, закрылся стартап или продолжает действовать. 
Временной период - '1970-01-01' по '2018-01-01'. 
Дата формирования выгрузки - '2018-01-01'

- `kaggle_startups_train_27042024.csv` - информация (53 000) стартапах, которые будут использоваться в качестве обучающих данных.
- `kaggle_startups_test_27042024.csv` - информация (13 000) стартапах, которые будут использоваться в качестве тестовых данных. Ваша задача - предсказать значение 'status' для каждого стартапа из этого датасета.
- `kaggle_startups_sample_submit_27042024.csv` - файл с примером предсказаний в правильном формате.
name - идентификатор (название) стартапа в тестовом наборе.
status - целевой признак. Для каждого стартапа предскажите категориальное значение соответствующее прогнозу ['operating', 'closed']. 

**Описание полей данных**
- `name` - Название стартапа
- `category_list` - Список категорий, к которым относится стартап
- `funding_total_usd` - Общая сумма финансирования в USD
- `status` - Статус стартапа (закрыт или действующий)
- `country_code` - Код страны
- `state_code` - Код штата
- `region` - Регион
- `city` - Город
- `funding_rounds` - Количество раундов финансирования
- `founded_at` - Дата основания
- `first_funding_at` - Дата первого раунда финансирования
- `last_funding_at` - Дата последнего раунда финансирования
- `closed_at` - Дата закрытия стартапа (если применимо)
- `lifetime` - Время существования стартапа в днях

In [68]:
! pip install phik -q
! pip install shap -q
! pip install -U scikit-learn -q
! pip install seaborn --upgrade -q
! pip install missingno -q

%config InlineBackend.figure_formats = ['svg']
%matplotlib inline



In [69]:
import pandas as pd
import os
import missingno as msno
import matplotlib.pyplot as plt

In [70]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.3f}'.format)

In [71]:
def preprocessing(df):
    print(f'Изучение данных датафрейма "\n')
    print(f'Вывод первых 5 строк \n')
    display(df.head(5))
    print('-'*100)
    print(f'Размерность датафрейма, составляет - {(df.shape)}')
    print('-'*100)
    print(f'Общая информация:\n')
    display(df.info())
    print('-'*100)
    print(f'Описательная статистика:\n')
    display(df.describe())
    print('-'*100)
    df.isna().sum()
    print(f'Проверка на наличие явных дубликатов:\n')
    display(df.duplicated().sum())
    print('-'*100)
    print(f'Вывод количества уникальных значений')
    display(pd.DataFrame(df.apply(lambda x: x.nunique())))

In [72]:
pth1 = './datasets/kaggle_startups_train_27042024.csv'
pth2 = './datasets/kaggle_startups_test_27042024.csv'

if os.path.exists(pth1):
    # Тренировочная выборка:
    df_ks_train = pd.read_csv(pth1)
    if os.path.exists(pth2):
        # Тестовая выборка:
        df_ks_test = pd.read_csv(pth2)
    else:
        print('Something is wrong')

In [73]:
preprocessing(df_ks_train)

Изучение данных датафрейма "

Вывод первых 5 строк 



,name,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at,closed_at
0,Lunchgate,Online Reservations|Restaurants,828626.000,operating,CHE,25,Zurich,Zürich,2,2009-12-31,2011-05-01,2014-12-01,NaN
1,EarLens,Manufacturing|Medical|Medical Devices,42935019.000,operating,USA,CA,SF Bay Area,Redwood City,4,2005-01-01,2010-05-04,2014-02-25,NaN
2,Reviva Pharmaceuticals,Biotechnology,35456381.000,operating,USA,CA,SF Bay Area,San Jose,3,2006-01-01,2012-08-20,2014-07-02,NaN
3,Sancilio and Company,Health Care,22250000.000,operating,NaN,NaN,NaN,NaN,3,2004-01-01,2011-09-01,2014-07-18,NaN
4,WireTough Cylinders,Manufacturing,NaN,operating,USA,VA,VA - Other,Bristol,1,2010-09-30,2012-02-01,2012-02-01,NaN


----------------------------------------------------------------------------------------------------
Размерность датафрейма, составляет - (52514, 13)
----------------------------------------------------------------------------------------------------
Общая информация:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52514 entries, 0 to 52513
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               52513 non-null  object 
 1   category_list      50049 non-null  object 
 2   funding_total_usd  42445 non-null  float64
 3   status             52514 non-null  object 
 4   country_code       47013 non-null  object 
 5   state_code         45752 non-null  object 
 6   region             46156 non-null  object 
 7   city               46156 non-null  object 
 8   funding_rounds     52514 non-null  int64  
 9   founded_at         52514 non-null  object 
 10  first_funding_at   52514 non-null  objec

None

----------------------------------------------------------------------------------------------------
Описательная статистика:



,funding_total_usd,funding_rounds
count,42445.000,52514.000
mean,18248255.980,1.741
std,187121675.710,1.375
min,1.000,1.000
25%,336000.000,1.000
50%,2000000.000,1.000
75%,10000000.000,2.000
max,30079503000.000,19.000


----------------------------------------------------------------------------------------------------
Проверка на наличие явных дубликатов:



0

----------------------------------------------------------------------------------------------------
Вывод количества уникальных значений


,0
name,52513
category_list,22108
funding_total_usd,15551
status,2
country_code,134
state_code,300
region,1036
city,4477
funding_rounds,19
founded_at,5401


In [82]:
nan_counts = df_ks_train.isna().sum()
print(nan_counts)

name                     1
category_list         2465
funding_total_usd    10069
status                   0
country_code          5501
state_code            6762
region                6358
city                  6358
funding_rounds           0
founded_at               0
first_funding_at         0
last_funding_at          0
closed_at            47599
dtype: int64


In [88]:
# Создание объекта осей
fig, ax = plt.subplots(figsize=(18, 4))
# Генерация графика с использованием объекта осей
msno.matrix(df_ks_train, ax=ax, sparkline=False)
# Добавление названия графика
ax.set_title('Матрица пропущенных значений')
# Отображение графика
plt.show()